In [1]:
import numpy as np
import pandas as pd
from sklearn import neighbors
from sklearn.cluster import KMeans

In [2]:
%%time
train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

Wall time: 38.5 s


In [3]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=1 #GridSearchをするかコントロール
SEED=12345
sample_num=10000
fold_num=5

#train関連
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_drop_col=['ID_code']

In [4]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

特徴量作成コードを記載

In [5]:
train_df.head()

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [84]:
train_df_0=train_df[train_df['target']==0].drop(['target','ID_code'],axis=1)
train_df_1=train_df[train_df['target']==1].drop(['target','ID_code'],axis=1)

In [89]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
train_T_sort=train_df_0[0:1].T
train_T_sort=train_T_sort.sort_values(by=[0], ascending=True)
train_T_sort['col_0']=train_T_sort.index
train_T_sort=train_T_sort.reset_index()

for i in range(1,10):
    train_T=train_df_0[i:(i+1)].T
    train_T['val']=train_df_0[i:(i+1)].T
    train_T=train_T.sort_values(by='val')
    train_T['col_'+str(i)]=train_T.index
    #print(train_T)
    train_T=train_T.reset_index()
    
    train_T_sort[str(i)]=train_T['val']
    train_T_sort['col_'+str(i)]=train_T['col_'+str(i)]
train_T_sort

,index,0,col_0,1,col_1,2,col_2,3,col_3,4,col_4,5,col_5,6,col_6,7,col_7,8,col_8,9,col_9
0,var_90,-21.4494,var_90,-47.3797,var_45,-22.4038,var_90,-35.1659,var_47,-65.4863,var_45,-44.7257,var_45,-29.9763,var_187,-27.2543,var_45,-31.7855,var_47,-39.3042,var_45
1,var_187,-19.7159,var_187,-15.9319,var_187,-17.2738,var_118,-25.0222,var_61,-39.7997,var_90,-24.6935,var_47,-22.2159,var_100,-24.7167,var_90,-18.3172,var_187,-27.7693,var_61
2,var_147,-16.4727,var_147,-15.5849,var_17,-13.5174,var_40,-24.6840,var_45,-27.7439,var_61,-18.1194,var_117,-13.2871,var_134,-14.4815,var_54,-18.0895,var_61,-21.6070,var_187
3,var_47,-14.2136,var_47,-14.9100,var_182,-13.2648,var_182,-19.8592,var_100,-22.9264,var_100,-16.8151,var_157,-12.6687,var_61,-12.0219,var_40,-16.4566,var_135,-17.9593,var_40
4,var_118,-13.4221,var_118,-13.6950,var_100,-11.7218,var_135,-15.2398,var_30,-12.7047,var_147,-13.6509,var_185,-12.3890,var_30,-11.1786,var_176,-14.4918,var_26,-16.2859,var_5
5,var_157,-13.1324,var_157,-13.5901,var_86,-11.2648,var_11,-12.8279,var_187,-12.6068,var_157,-9.0953,var_167,-10.0345,var_157,-10.9163,var_167,-14.2764,var_157,-13.8960,var_178
6,var_176,-10.1245,var_176,-11.7684,var_26,-11.1025,var_30,-11.0882,var_164,-12.1419,var_11,-8.9114,var_187,-9.2271,var_90,-10.6657,var_147,-11.4027,var_182,-13.1141,var_30
7,var_5,-9.2834,var_5,-11.5100,var_118,-10.3734,var_61,-10.9370,var_155,-9.4458,var_10,-8.8256,var_134,-8.1416,var_180,-10.5909,var_82,-11.1344,var_30,-12.3881,var_100
8,var_171,-8.4254,var_171,-11.1147,var_40,-9.7009,var_178,-10.4701,var_178,-9.2587,var_82,-8.3794,var_100,-8.0198,var_5,-8.4758,var_65,-9.9819,var_90,-10.6731,var_97
9,var_180,-8.4051,var_180,-9.6953,var_157,-9.0837,var_5,-8.9996,var_199,-8.8104,var_199,-8.3058,var_84,-7.5271,var_11,-8.4732,var_52,-9.5251,var_167,-10.5564,var_147


In [90]:
train_T_sort['val']=train_df_1[0:1].T
train_T_sort=train_T_sort.sort_values(by='val', ascending=True)
train_T_sort['col_0']=train_T_sort.index
train_T_sort=train_T_sort.reset_index()

for i in range(1,10):
    train_T=train_df_1[i:(i+1)].T
    train_T['val']=train_df_1[i:(i+1)].T
    train_T=train_T.sort_values(by='val')
    train_T['col_'+str(i)]=train_T.index
    #print(train_T)
    train_T=train_T.reset_index()
    
    train_T_sort[str(i)]=train_T['val']
    train_T_sort['col_'+str(i)]=train_T['col_'+str(i)]
train_T_sort

,level_0,index,0,col_0,1,col_1,2,col_2,3,col_3,4,col_4,5,col_5,6,col_6,7,col_7,8,col_8,9,col_9,val
0,0,var_90,-21.4494,0,-21.7322,var_187,-33.7053,var_45,-33.5923,var_90,-27.7489,var_90,-29.2175,var_90,-29.1320,var_90,-22.8877,var_47,-21.7059,var_187,-30.3886,var_187,NaN
1,1,var_187,-19.7159,1,-18.6567,var_100,-17.1437,var_47,-23.8759,var_45,-18.4018,var_61,-26.1386,var_61,-23.1345,var_61,-16.8102,var_100,-18.9158,var_47,-22.2960,var_90,NaN
2,2,var_147,-16.4727,2,-16.8994,var_135,-14.6391,var_30,-18.7344,var_100,-15.0248,var_47,-19.2400,var_167,-21.2075,var_47,-16.5813,var_118,-14.3487,var_167,-19.9356,var_199,NaN
3,3,var_47,-14.2136,3,-16.2435,var_30,-13.6216,var_176,-17.6521,var_61,-14.9327,var_178,-17.8708,var_199,-20.9520,var_176,-14.8816,var_199,-14.2724,var_61,-19.3989,var_45,NaN
4,4,var_118,-13.4221,4,-15.7092,var_40,-13.3627,var_5,-16.6872,var_30,-11.7573,var_69,-16.4645,var_5,-14.0964,var_147,-14.2796,var_82,-13.0134,var_100,-16.4207,var_118,NaN
5,5,var_157,-13.1324,5,-14.6897,var_199,-13.0577,var_187,-15.8005,var_135,-11.4237,var_10,-15.6440,var_30,-13.0600,var_155,-14.2142,var_17,-12.4023,var_82,-15.9805,var_135,NaN
6,6,var_176,-10.1245,6,-13.3660,var_134,-12.7515,var_84,-13.6810,var_47,-9.0880,var_76,-12.1196,var_135,-10.5154,var_157,-10.4353,var_40,-11.9799,var_118,-15.5650,var_17,NaN
7,7,var_5,-9.2834,7,-12.6789,var_26,-12.6889,var_61,-12.7057,var_147,-8.6036,var_11,-11.2541,var_45,-10.1016,var_26,-10.3256,var_86,-11.2830,var_199,-12.6491,var_182,NaN
8,8,var_171,-8.4254,8,-10.3910,var_171,-11.7853,var_157,-10.7504,var_10,-7.1606,var_30,-10.6847,var_176,-10.0054,var_187,-8.8189,var_180,-10.6699,var_26,-10.9759,var_52,NaN
9,9,var_180,-8.4051,9,-9.7420,var_147,-10.5073,var_52,-9.5054,var_149,-6.0641,var_80,-10.2170,var_185,-9.9429,var_52,-8.5400,var_52,-9.3716,var_147,-10.3615,var_157,NaN


In [ ]:
train_df.to_csv('./feature/102_train_kmeans.csv',index=False)
test_df.to_csv('./feature/102_test_kmeans.csv',index=False)